# Projeto de Limpeza e Tratamento de Dados – Seguro Rural

Este notebook documenta o processo de **preparação, padronização e qualificação dos dados** referentes ao Programa de Subvenção ao Prêmio do Seguro Rural (PSR), abrangendo os períodos de **2006 a 2025**.  
O objetivo central é consolidar múltiplas bases históricas em uma **tabela única, íntegra e confiável**, garantindo consistência e coerência para análises futuras.

## Etapas tratadas:
- **Carregamento e avaliação inicial** das diferentes bases CSV (2006–2015, 2016–2024 e 2025).  
- **Comparação estrutural** entre datasets, com padronização de tipos de dados divergentes.  
- **Empilhamento das bases** e criação de uma única tabela consolidada com mais de 1,7 milhão de registros.  
- **Tratamento de duplicatas** e verificação de chaves primárias.  
- **Conversão de tipos de dados** (numéricos, datas e textos).  
- **Análises de consistência temporal** (vigência, proposta e emissão de apólice).  
- **Tratamento de nulos, outliers e redundâncias textuais** em variáveis categóricas.  
- **Criação de flags de inconsistência** para suporte a análises de qualidade de dados.  
- **Exportação da base limpa** em formato otimizado (Parquet), pronta para análises estatísticas e modelagem.  

## Soluções adotadas:
- Padronização de colunas divergentes para **tipos comuns entre datasets**.  
- Criação de **funções genéricas** para conversão de datas e padronização de texto.  
- Uso de **validações temporais e estatísticas** para identificar inconsistências.  
- Aplicação de **regras de negócio** específicas para o setor de seguros rurais, como limites de vigência, coerência entre produtividade estimada e segurada e plausibilidade dos valores de subvenção federal.  


In [ ]:
# Importações
import pandas as pd

## Avaliação inicial dos datasets

In [ ]:
# Carregar o CSV dados_abertos_psr_2016a2024
df = pd.read_csv(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\raw\dados_abertos_psr_2016a2024csv.csv", sep=";", encoding="latin1", low_memory=False)

# Exibir as primeiras linhas para conferir
print(df.head())

# Ver informações gerais
print(df.info())


In [ ]:
# Carregar o CSV dados_abertos_psr_2006a2015
df1 = pd.read_csv(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\raw\dados_abertos_psr_2006a2015csv.csv", sep=";", encoding="latin1", low_memory=False)

# Exibir as primeiras linhas para conferir
print(df1.head())

# Ver informações gerais
print(df1.info())

In [ ]:
# Carregar o CSV dados_abertos_psr_2025
df2 = pd.read_csv(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\raw\dados_abertos_psr_2025csv.csv", sep=";", encoding="latin1", low_memory=False)

# Exibir as primeiras linhas para conferir
print(df2.head())

# Ver informações gerais
print(df2.info())

## Tratamento para criação de tabela única

In [ ]:
# Verificação de tipos de dados em colunas para identificar divergências entre os datasets
import itertools
import pandas as pd

def comparar_estruturas(*dfs, nomes=None):
    if nomes is None:
        nomes = [f"df{i}" for i in range(len(dfs))]

    for (i, df_a), (j, df_b) in itertools.combinations(enumerate(dfs), 2):
        nome_a, nome_b = nomes[i], nomes[j]
        print(f"\n🔎 Comparando {nome_a} x {nome_b}:")
        
        # Colunas
        if not df_a.columns.equals(df_b.columns):
            print("❌ Colunas diferentes:")
            print(f"No {nome_a} e não no {nome_b}: ", set(df_a.columns) - set(df_b.columns))
            print(f"No {nome_b} e não no {nome_a}: ", set(df_b.columns) - set(df_a.columns))
        else:
            print("✅ Mesmos nomes e ordem das colunas")
        
        # Dtypes
        diff_dtypes = (df_a.dtypes != df_b.dtypes)
        if diff_dtypes.any():
            print("⚠️ Tipos diferentes nestas colunas:")
            print(pd.concat([df_a.dtypes, df_b.dtypes], axis=1, keys=[nome_a,nome_b])[diff_dtypes])
        else:
            print("✅ Mesmos tipos de dados")

# Exemplo de uso
comparar_estruturas(df, df1, df2, nomes=["df", "df1", "df2"])



In [ ]:
# Padronização das colunas divergentes

cols_para_object = ["CD_PROCESSO_SUSEP", "NR_AREA_TOTAL"]

# Converte no df1
df1[cols_para_object] = df1[cols_para_object].astype("object")

# Conferir resultado
print(df1.dtypes.loc[cols_para_object])


In [ ]:
# Re-avaliação para constatar a padronização
comparar_estruturas(df, df1, df2, nomes=["df", "df1", "df2"])

In [ ]:
# Empilhamento dos datasets para criação de base única
df_raw = pd.concat([df, df1, df2], ignore_index=True)


In [ ]:
# Conferir resultado
print(df_raw.head())

In [ ]:
# Conferir quantidade de registros
print(df.shape)
print(df1.shape)
print(df2.shape)
print(df_raw.shape)


## Análise de conscistências da tabela

### Verificação de nulos

In [ ]:
# Avaliação de quantidade de nulos por coluna
for col in df_raw.columns:
    print(col, (round(df_raw[col].isnull().sum()*100 /len(df_raw),2)))

### Resumo sobre valores nulos
A análise de valores ausentes revelou **baixa ocorrência em colunas-chave**, concentrando-se principalmente nas coordenadas geográficas e na variável **NR_ANIMAL**, com aproximadamente **47% de nulos**. Essa ausência é esperada, uma vez que a cobertura animal não se aplica a todas as apólices do seguro rural. Nas variáveis de latitude e longitude, os nulos estão distribuídos de forma marginal (até 1,7%), permitindo imputações ou descartes controlados conforme a necessidade analítica.  

### Verificação de duplicatas

In [ ]:
# Identificação de chave primária 
# Verificação de registros duplicados
duplicados = df_raw['ID_PROPOSTA'].duplicated().sum()

if duplicados == 0:
    print("✅ ID_PROPOSTA é chave única")
else:
    print(f"❌ Existem {duplicados} duplicados em ID_PROPOSTA")


In [ ]:
# Identificação de registro duplicado
contagem = df_raw['ID_PROPOSTA'].value_counts()
print(contagem[contagem > 1])


In [ ]:
# Identificação do id
df_raw[df_raw['ID_PROPOSTA'] == 1956479]


In [ ]:
# Extração da duplicata
df_sem_duplicatas = df_raw.drop(553244)


In [ ]:
# Re-avaliação de presença de duplicata
contagem = df_sem_duplicatas['ID_PROPOSTA'].value_counts()
print(contagem[contagem > 1])

### Resumo sobre duplicatas
Foi identificada a duplicidade do **ID_PROPOSTA 1956479**, relacionada a registros da mesma apólice, porém vinculados a municípios diferentes. Após análise, um dos registros foi descartado, resultando em uma base sem duplicatas no identificador primário, garantindo a integridade da chave.

### Conversão de tipo de dado das colunas

In [ ]:
# Conversão de dados numéricos para tipos coerentes
cols_para_int = ['NR_GRAU_LAT', 'NR_MIN_LAT', 'NR_SEG_LAT',
                 'NR_GRAU_LONG', 'NR_MIN_LONG', 'NR_SEG_LONG',
                 'NR_ANIMAL']

for col in cols_para_int:
    df_raw[col] = pd.to_numeric(df_sem_duplicatas[col], errors='coerce').astype('Int64')  # Int64 aceita NaN

cols_para_float = ['NR_AREA_TOTAL', 'NR_PRODUTIVIDADE_ESTIMADA',
                   'NR_PRODUTIVIDADE_SEGURADA', 'VL_LIMITE_GARANTIA',
                   'VL_PREMIO_LIQUIDO', 'PE_TAXA',
                   'VL_SUBVENCAO_FEDERAL', 'VALOR_INDENIZAÇÃO']

for col in cols_para_float:
    df_sem_duplicatas[col] = pd.to_numeric(df_sem_duplicatas[col], errors='coerce')



In [ ]:
df_sem_duplicatas['NR_GRAU_LAT'].head()

In [ ]:
df_sem_duplicatas['DT_PROPOSTA'].head()

In [ ]:
# Função para conversão de colunas com formato data
import pandas as pd

def converter_datas(df, cols, formato_brasil=True):
    """Converte colunas de data para datetime64.
       Substitui valores inválidos por NaT e mostra registros problemáticos.
       
       Args:
           df (pd.DataFrame): DataFrame original
           cols (list): Lista de colunas a converter
           formato_brasil (bool): True se formato for dd/mm/yyyy
       Returns:
           pd.DataFrame: DataFrame com colunas convertidas
    """
    for col in cols:
        # Converter
        df[col] = pd.to_datetime(
            df[col],
            errors="coerce",
            dayfirst=formato_brasil
        )
        
        total = len(df)
        n_invalidos = df[col].isna().sum()
        print(f"Coluna {col}: convertida ✅ ({n_invalidos}/{total} inválidos)")
        
        # Mostrar registros inválidos
        if n_invalidos > 0:
            print(f"🔎 Registros inválidos na coluna {col}:")
            print(df.loc[df[col].isna(), col].head(10))  # Mostra até 10 para não explodir a tela
            print("----")
    return df



In [ ]:
# Execução da função converter_datas
colunas_data = ["DT_PROPOSTA", "DT_INICIO_VIGENCIA", "DT_FIM_VIGENCIA", "DT_APOLICE"]
df_sem_duplicatas = converter_datas(df_sem_duplicatas, colunas_data)


### Resumo sobre conversão de tipos de dados
Diversas colunas inicialmente armazenadas como `object` foram convertidas para **tipos numéricos e temporais adequados**.  
- Colunas como área, produtividade, valores monetários e taxas foram convertidas para **float**.  
- Datas (proposta, vigência e apólice) foram transformadas para `datetime64`, permitindo análises temporais.  
Esse ajuste melhora a confiabilidade estatística e a performance dos cálculos posteriores.

### Análise de coerência das datas

In [ ]:
# Análise com referência da Data da Apólice
df_sem_duplicatas['DIAS_APOLICE_DEPOIS'] = (
    df_sem_duplicatas['DT_APOLICE'] - df_sem_duplicatas['DT_INICIO_VIGENCIA']
).dt.days

# Criar flag de inconsistência
df_sem_duplicatas['APOLICE_INCONSISTENTE'] = (
    (df_sem_duplicatas['DIAS_APOLICE_DEPOIS'] < 0) |
    (df_sem_duplicatas['DIAS_APOLICE_DEPOIS'] > 60)
)

# Relatório resumido
total = len(df_sem_duplicatas)
problemas = df_sem_duplicatas['APOLICE_INCONSISTENTE'].sum()
ok = total - problemas

print("📊 Validação das datas de apólice vs início de vigência")
print("-" * 60)
print(f"Total de registros analisados: {total:,}")
print(f"✅ Registros dentro da regra (0 a 60 dias): {ok:,} ({ok/total:.2%})")
print(f"⚠️ Registros inconsistentes: {problemas:,} ({problemas/total:.2%})")
print("-" * 60)

# Se quiser inspecionar os primeiros inconsistentes
print("Exemplos de inconsistências:")
print(df_sem_duplicatas.loc[df_sem_duplicatas['APOLICE_INCONSISTENTE'],
                           ['ID_PROPOSTA','DT_PROPOSTA','DT_APOLICE','DT_INICIO_VIGENCIA','DT_FIM_VIGENCIA','DIAS_APOLICE_DEPOIS']].head(10))


In [ ]:
# Análise sem a referência da Data da Apólice
df_sem_duplicatas["DIAS_PROPOSTA_ANTES_VIGENCIA"] = (
    (df_sem_duplicatas["DT_INICIO_VIGENCIA"] - df_sem_duplicatas["DT_PROPOSTA"]).dt.days
)

df_sem_duplicatas["DIAS_VIGENCIA"] = (
    (df_sem_duplicatas["DT_FIM_VIGENCIA"] - df_sem_duplicatas["DT_INICIO_VIGENCIA"]).dt.days
)

# Regras de consistência
df_sem_duplicatas["ERRO_PROPOSTA"] = df_sem_duplicatas["DIAS_PROPOSTA_ANTES_VIGENCIA"] < 0
df_sem_duplicatas["ERRO_VIGENCIA"] = df_sem_duplicatas["DIAS_VIGENCIA"] <= 0
df_sem_duplicatas["ERRO_VIGENCIA_EXCESSO"] = df_sem_duplicatas["DIAS_VIGENCIA"] > 730  # mais de 2 anos

# Relatório
total = len(df_sem_duplicatas)
erros_proposta = df_sem_duplicatas["ERRO_PROPOSTA"].sum()
erros_vigencia = df_sem_duplicatas["ERRO_VIGENCIA"].sum()
erros_excesso = df_sem_duplicatas["ERRO_VIGENCIA_EXCESSO"].sum()

print("📊 Validação de DT_PROPOSTA, DT_INICIO_VIGENCIA e DT_FIM_VIGENCIA")
print("-" * 60)
print(f"Total de registros analisados: {total:,}")
print(f"⚠️ Proposta depois do início da vigência: {erros_proposta:,} ({erros_proposta/total:.2%})")
print(f"⚠️ Vigência com datas invertidas (fim <= início): {erros_vigencia:,} ({erros_vigencia/total:.2%})")
print(f"⚠️ Vigência excessiva (> 2 anos): {erros_excesso:,} ({erros_excesso/total:.2%})")
print("-" * 60)

# Mostrar exemplos problemáticos
print("Exemplos de registros inconsistentes:")
print(df_sem_duplicatas.loc[
    df_sem_duplicatas["ERRO_PROPOSTA"] | df_sem_duplicatas["ERRO_VIGENCIA"] | df_sem_duplicatas["ERRO_VIGENCIA_EXCESSO"],
    ["ID_PROPOSTA", "DT_PROPOSTA", "DT_INICIO_VIGENCIA", "DT_FIM_VIGENCIA",
     "DIAS_PROPOSTA_ANTES_VIGENCIA", "DIAS_VIGENCIA"]
].head(10))


### Resumo sobre coerência temporal
Foram avaliadas as regras de consistência entre **datas de proposta, início e fim de vigência e data de apólice**.  
- Cerca de **36% dos registros apresentaram vigência inválida** (data final menor ou igual à inicial).  
- Aproximadamente **46% apresentaram diferenças excessivas** entre emissão da apólice e início da vigência.  
Essas inconsistências, ainda que não impossibilitem análises, requerem tratamento em fases posteriores (por exemplo, exclusão ou ajuste de registros incoerentes).


### Tratamento de padronização de texto

In [ ]:
# Identificar colunas de texto (object ou string)
cols_str = df_sem_duplicatas.select_dtypes(include=['object', 'string']).columns

for col in cols_str:
    # Converter para string (garantir)
    df_sem_duplicatas[col] = df_sem_duplicatas[col].astype(str)
    
    # Passo 1: tudo minúsculo
    df_sem_duplicatas[col] = df_sem_duplicatas[col].str.lower()
    
    # Passo 2: remover espaços extras no início e fim
    df_sem_duplicatas[col] = df_sem_duplicatas[col].str.strip()
    
    # Passo 3: substituir espaços internos por underscore
    df_sem_duplicatas[col] = df_sem_duplicatas[col].str.replace(r"\s+", "_", regex=True)
    
    # Passo 4: substituir múltiplos underscores seguidos por apenas 1
    df_sem_duplicatas[col] = df_sem_duplicatas[col].str.replace(r"_+", "_", regex=True)

print(f"✅ Colunas de texto tratadas: {list(cols_str)}")


In [ ]:
# Avaliação de textos redundantes
valores_evento = df_sem_duplicatas['EVENTO_PREPONDERANTE'].unique()
valores_cultura = df_sem_duplicatas['NM_CULTURA_GLOBAL'].unique()

print("📋 Valores únicos de EVENTO_PREPONDERANTE:")
print(valores_evento)

print("\n📋 Valores únicos de NM_CULTURA_GLOBAL:")
print(valores_cultura)


### Resumo sobre padronização de texto
As colunas categóricas foram padronizadas para:  
- Texto em minúsculo.  
- Substituição de espaços por underscores.  
- Remoção de redundâncias e caracteres especiais.  

Isso garante maior consistência nas análises e facilita agrupamentos e filtros.  




### Verificação de outliers

In [ ]:
# 1. Regras manuais de inconsistência
regras = (
    (df_sem_duplicatas["VL_PREMIO_LIQUIDO"] == 0) |
    (df_sem_duplicatas["VL_SUBVENCAO_FEDERAL"] < 0)  # se não deveria ser negativo
)

# 2. Detectar outliers em VL_SUBVENCAO_FEDERAL pelo critério IQR (boxplot)
Q1 = df_sem_duplicatas["VL_SUBVENCAO_FEDERAL"].quantile(0.25)
Q3 = df_sem_duplicatas["VL_SUBVENCAO_FEDERAL"].quantile(0.75)
IQR = Q3 - Q1

limite_superior = Q3 + 1.5 * IQR

outliers_subv = df_sem_duplicatas["VL_SUBVENCAO_FEDERAL"] > limite_superior

# 3. Consolidar inconsistências
df_inconsistencias = df_sem_duplicatas.loc[
    regras | outliers_subv,
    ["ID_PROPOSTA", "VL_PREMIO_LIQUIDO", "VL_SUBVENCAO_FEDERAL"]
]

print("📊 Registros com valores fora do esperado:")
print(df_inconsistencias.head(20))

# Estatísticas de apoio
print("\nResumo estatístico de VL_SUBVENCAO_FEDERAL:")
print(df_sem_duplicatas["VL_SUBVENCAO_FEDERAL"].describe())
print(f"Limite superior considerado para outlier: {limite_superior:.2f}")


In [ ]:
# Calcular métricas de comparação
df_sem_duplicatas["DIF_PROD"] = (
    df_sem_duplicatas["NR_PRODUTIVIDADE_SEGURADA"] - df_sem_duplicatas["NR_PRODUTIVIDADE_ESTIMADA"]
)

df_sem_duplicatas["RAZAO_PROD"] = (
    df_sem_duplicatas["NR_PRODUTIVIDADE_SEGURADA"] / df_sem_duplicatas["NR_PRODUTIVIDADE_ESTIMADA"]
)

# Regras de inconsistência
inconsistencias_prod = df_sem_duplicatas[
    (df_sem_duplicatas["RAZAO_PROD"] < 0.8) |   # muito abaixo
    (df_sem_duplicatas["RAZAO_PROD"] > 1.2)     # muito acima
]

print("📊 Registros onde a produtividade segurada foge muito da estimada:")
print(inconsistencias_prod[["ID_PROPOSTA","NR_PRODUTIVIDADE_ESTIMADA","NR_PRODUTIVIDADE_SEGURADA","DIF_PROD","RAZAO_PROD"]].head(20))

# Estatísticas
print("\nResumo da razão segurada/estimada:")
print(df_sem_duplicatas["RAZAO_PROD"].describe())


### Resumo sobre outliers
Foram aplicadas regras manuais e estatísticas (IQR) para detectar **outliers em variáveis monetárias e de produtividade**.  
- Alguns registros apresentaram valores de subvenção acima do limite plausível.  
- Diferenças significativas entre **produtividade estimada** e **segurada** foram identificadas em milhares de casos, geralmente com desvio de 30% ou mais.  
Esses registros foram sinalizados com **flags**, preservando a base íntegra mas permitindo exclusões ou ajustes nas análises posteriores.


In [ ]:
# Salvar em Parquet
df_sem_duplicatas.to_parquet(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\interim\df_interim.parquet", index=False)

print("✅ Dataset salvo em Parquet com sucesso!")


### Resumo final das verificações
- **Dados textuais** padronizados.  
- **Datas e numéricos** ajustados para tipos coerentes.  
- **Duplicatas eliminadas** e chaves consistentes.  
- **Outliers e inconsistências temporais sinalizados** com flags de qualidade.  
- **Base consolidada** em Parquet com mais de 1,7 milhão de registros válidos.  

# Parecer Final – Etapa de Limpeza e Tratamento

A consolidação e preparação da base de **Seguro Rural (2006–2025)** foi concluída com sucesso, resultando em um dataset robusto e padronizado, adequado para análises avançadas no contexto do agronegócio.  

### Principais conquistas:
- Unificação de diferentes períodos históricos em uma **base única e consistente**.  
- Correção de divergências estruturais e de tipos de dados.  
- Identificação e eliminação de duplicatas críticas.  
- Criação de flags de qualidade para **monitorar nulos, outliers e incoerências temporais**.  
- Exportação para formato otimizado (Parquet), pronto para integração em pipelines analíticos.  

### Recomendações futuras:
1. **Tratar inconsistências temporais** (36% de vigências inválidas e 46% de apólices fora da janela esperada).  
2. **Revisar outliers críticos** em variáveis monetárias e de produtividade, avaliando se representam erros de registro ou fenômenos do setor.  
3. **Aplicar técnicas de detecção de anomalias** em séries temporais e análises preditivas para maior confiabilidade.  

Em resumo, esta etapa entrega uma **fundação sólida** para análises estatísticas, modelos preditivos e dashboards gerenciais, sustentando decisões estratégicas relacionadas à política agrícola e ao seguro rural.
